In [1]:
import pandas as pd
import numpy as np
import matplotlib


In [2]:
df0 = pd.read_csv('datos.txt', skiprows=1, sep=',',header=None,names=["TIMESTAMP","RECORD","batt_volt_Min","logger_temp","short_up_Avg","short_dn_Avg","long_up_Avg","long_dn_Avg","cnr4_T_C_Avg","cnr4_T_K_Avg","long_up_corr_Avg","long_dn_corr_Avg","Rs_net_Avg","Rl_net_Avg","albedo_Avg","Rn_Avg","AirTC_Avg","RH","WS_ms_S_WVT","WindDir_D1_WVT","WindDir_SD1_WVT","Temp_C_Avg(1)","Temp_C_Avg(2)","Temp_C_Avg(3)","Temp_C_Avg(4)","Result1_Avg","Result2_Avg","Result3_Avg","Result4_Avg","Result5_Avg","Result6_Avg","Result7_Avg","Result8_Avg","Result9_Avg","Riego_mm_Tot","Perco_mm_Tot"])

In [3]:
df0['TIMESTAMP'] = pd.to_datetime(df0.apply(lambda x: x['TIMESTAMP'], 1), dayfirst=True)
df0 = pd.DataFrame(df0)

In [4]:
#Solo nos interesan 2 columnas  TIMESTAMP (Fecha) y  AirTC_Avg (Temperatura del aire, ºC) 
df0 = df0[['TIMESTAMP','AirTC_Avg']]

In [5]:
df0.dtypes 

TIMESTAMP    datetime64[ns]
AirTC_Avg           float64
dtype: object

In [6]:
#Se crea una columna para las horas, dias, meses y años
df0['Hora']=df0['TIMESTAMP'].dt.hour
df0['Dia']=df0['TIMESTAMP'].dt.day
df0['Mes']=df0['TIMESTAMP'].dt.month
df0['Año']=df0['TIMESTAMP'].dt.year

In [7]:
#Ahora se saca la temperatura media por hora desde el mes que de pide en la actividad
#que en este caso es NOVIEMBRE
dfmedia=df0.set_index('TIMESTAMP').resample('H').mean()


In [8]:
#Se agarran los valores desde noviembre hasta el ultimo dato
dfmedia = dfmedia.loc['2018-11-01 00:00:00':'2019-02-18 07:30:00']

In [9]:
conditions = [
    (dfmedia['AirTC_Avg'] >= 1.5) & (dfmedia['AirTC_Avg'] <= 2.4),
    (dfmedia['AirTC_Avg'] >= 2.5) & (dfmedia['AirTC_Avg'] <= 9.1),
    (dfmedia['AirTC_Avg'] >= 9.2) & (dfmedia['AirTC_Avg'] <= 12.4),
    (dfmedia['AirTC_Avg'] >= 12.5) & (dfmedia['AirTC_Avg'] <= 15.9),
    (dfmedia['AirTC_Avg'] >= 16) & (dfmedia['AirTC_Avg'] <=18),
    (dfmedia['AirTC_Avg'] > 18 )]

choices = [0.5, 1.0, 0.5, 0, -0.5, -1.0]

dfmedia['UF'] = np.select(conditions, choices, default=0)

dfmedia['UF24'] = dfmedia.groupby(['Año','Mes','Dia'])['UF'].transform('sum')

In [10]:
df10 = dfmedia[['AirTC_Avg']]

In [11]:
df10 = df10.resample('D').sum()

In [12]:
df10 ['UF'] = dfmedia['UF24']

#agremamos una nueva columna TMAX que nos motrara la temperastura maxima

dfmedia ['TMAX'] = df0.set_index('TIMESTAMP').resample('D')["AirTC_Avg"].max()

In [13]:
#se muestra la tabla
dfmedia

,AirTC_Avg,Hora,Dia,Mes,Año,UF,UF24,TMAX
TIMESTAMP,,,,,,,,
2018-11-01 00:00:00,8.708333,0.0,1.0,11.0,2018.0,1.0,-0.5,29.580
2018-11-01 01:00:00,8.493333,1.0,1.0,11.0,2018.0,1.0,-0.5,NaN
2018-11-01 02:00:00,8.690000,2.0,1.0,11.0,2018.0,1.0,-0.5,NaN
2018-11-01 03:00:00,8.846667,3.0,1.0,11.0,2018.0,1.0,-0.5,NaN
2018-11-01 04:00:00,7.397500,4.0,1.0,11.0,2018.0,1.0,-0.5,NaN
2018-11-01 05:00:00,7.289833,5.0,1.0,11.0,2018.0,1.0,-0.5,NaN
2018-11-01 06:00:00,6.806833,6.0,1.0,11.0,2018.0,1.0,-0.5,NaN
2018-11-01 07:00:00,8.110167,7.0,1.0,11.0,2018.0,1.0,-0.5,NaN
2018-11-01 08:00:00,14.960000,8.0,1.0,11.0,2018.0,0.0,-0.5,NaN


In [14]:
df10 ['UF'] = dfmedia['UF24']

#agremamos una otra columna TMIN que nos mostrara la tempreratura minima

dfmedia ['TMIN'] = df0.set_index('TIMESTAMP').resample('D')["AirTC_Avg"].min()

In [15]:
#se muestra la tabla
dfmedia

,AirTC_Avg,Hora,Dia,Mes,Año,UF,UF24,TMAX,TMIN
TIMESTAMP,,,,,,,,,
2018-11-01 00:00:00,8.708333,0.0,1.0,11.0,2018.0,1.0,-0.5,29.580,6.096
2018-11-01 01:00:00,8.493333,1.0,1.0,11.0,2018.0,1.0,-0.5,NaN,NaN
2018-11-01 02:00:00,8.690000,2.0,1.0,11.0,2018.0,1.0,-0.5,NaN,NaN
2018-11-01 03:00:00,8.846667,3.0,1.0,11.0,2018.0,1.0,-0.5,NaN,NaN
2018-11-01 04:00:00,7.397500,4.0,1.0,11.0,2018.0,1.0,-0.5,NaN,NaN
2018-11-01 05:00:00,7.289833,5.0,1.0,11.0,2018.0,1.0,-0.5,NaN,NaN
2018-11-01 06:00:00,6.806833,6.0,1.0,11.0,2018.0,1.0,-0.5,NaN,NaN
2018-11-01 07:00:00,8.110167,7.0,1.0,11.0,2018.0,1.0,-0.5,NaN,NaN
2018-11-01 08:00:00,14.960000,8.0,1.0,11.0,2018.0,0.0,-0.5,NaN,NaN


In [16]:
count=[]
for i in range (0,len(df0))
z=df0.TEMP[i]
if(z<1.4):
    j=0
    if(z>1.5 and z<=2.4):
    j=0.5
    
    if(z>2.5 and z<=9.1):
    j=1
    if(z>9.2 and z<=12.4):
    j=0.5
    if(z>12.5 and z<=15.9):
    j=0
    if(z>16 and z<=18):
    j=-0.5
    if(z>18):
        j=-1
        count.append(j)
        df0["UF24"]=count

SyntaxError: invalid syntax (<ipython-input-16-f155c6ecd78d>, line 2)

In [17]:
df1=df0.drop(["AirTC_Avg","Hora","UF24"],1)
df1=df1.drop_duplicates()
f=len(df1)
df1.intex = np.arange(0,f)
df1.head()


df2=pd.DataFRame({})
df2["day"]=df1.Dia
df2["month"]=df1.Mes
df2["year"]=df1.Año
df2.intex = np.arange(0,f)
df2.head()
j=pd.to_datetime(df2)

df1["FECHA"]=j

sns.set(rc={'figure.figsize':(12,9)})

fig, ax = ply.subplots(figsize=(20, 10))


ax.plot(df0.FECHA, df0.TMAX, "o-")
ax.plot(df0.FECHA, df0.TMIN, "o-")
plt.title("Evolucion de temperatutas Minimas y Maximas", size=15)
plt.ylabel("°C", size=14)

fig.autofmt_xdate()

plt.show()

KeyError: "labels ['UF24'] not contained in axis"